Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
#@title  Añadir soporte GGUF a ComfyUI
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/city96/ComfyUI-GGUF.git
%cd $WORKSPACE/custom_nodes/ComfyUI-GGUF
!pip install -r requirements.txt



In [ ]:
#@title descargamos un modelo guff
%cd $WORKSPACE/models/unet
!wget -nc "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q3_K_S.gguf"

In [ ]:
#@title Descarga de modelos (CLIP) requeridos para DualCLIPLoader (GGUF) con nombres correctos
%cd $WORKSPACE

!mkdir -p models/clip
%cd models/clip

# Descargar CLIP y renombrar a clip-vit-large-patch14.bin
!wget -nc -O CLIP-GmP-ViT-L-14.safetensors "https://huggingface.co/zer0int/CLIP-GmP-ViT-L-14/resolve/main/model.safetensors?download=true"

# Descargar T5 y renombrar a model.safetensors
!wget -nc -O t5-v1_1-xxl-encoder-Q4_K_S.gguf "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf?download=true"

# Verifica los archivos
!ls -lh $WORKSPACE/models/clip/


In [ ]:
#@title descargamos el vae
%cd $WORKSPACE/models/vae
!wget https://huggingface.co/StableDiffusionVN/Flux/resolve/main/Vae/flux_vae.safetensors

In [ ]:
#@title opcional pero recomendad Easy Use
#1. Clone the repo
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/yolain/ComfyUI-Easy-Use
#2. Install the requirements
%cd $WORKSPACE/custom_nodes/ComfyUI-Easy-Use
!pip install -r requirements.txt


In [ ]:
#@title Run ComfyUI with cloudflared (Recommended Way)
%cd $WORKSPACE
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server